In [1]:
# import necessary BuildingMOTIF libraries
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BRICK # import this to make writing URIs easier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [2]:
# Create an in-memory BuildingMOTIF instance
bm = BuildingMOTIF("sqlite://")

In [3]:
# create the namespace for the building
BLDG = Namespace('urn:bldg/')

In [4]:
# Create the building model and add a description. Store the information in a variable "model"
model = Model.create(BLDG, description="This is a test model for a simple building")

In [5]:
# Print the current content of the ".ttl" shape graph in "model"
print(model.graph.serialize())

@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:bldg/> a owl:Ontology .




In [6]:
# load the full BRICK library into the variable "brick"
brick = Library.load(ontology_graph="Brick-full.ttl")

/Users/huangwp/opt/anaconda3/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: rkzeug is an invalid version and will not be supported in a future release
  warnings.warn(
2023-01-19 09:38:58,803 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject
2023-01-19 09:38:58,804 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice


In [7]:
# print the first 10 templates in the BRICK library
print("The Brick library contains the following templates:")
for template in brick.get_templates()[:10]:
    print(f"  - {template.name}")
    

The Brick library contains the following templates:
  - https://brickschema.org/schema/Brick#Methane_Level_Sensor
  - https://brickschema.org/schema/Brick#Discharge_Air_Temperature_Setpoint
  - https://brickschema.org/schema/Brick#CO2_Sensor
  - https://brickschema.org/schema/Brick#Entering_Condenser_Water_Temperature_Setpoint
  - https://brickschema.org/schema/Brick#Reset_Command
  - https://brickschema.org/schema/Brick#TimeShape
  - https://brickschema.org/schema/Brick#Fire_Alarm
  - https://brickschema.org/schema/Brick#Return_Air_Grains_Sensor
  - https://brickschema.org/schema/Brick#Elevator_Space
  - https://brickschema.org/schema/Brick#Meter


In [8]:
# First, we create an Excel table to express building equipment relations. This Excel table should consist of 5 columns.
# The first column is the "subject" equipment. The second column is the "subject" equipment's BRICK class.
# The third column is the BRICK relation, which can include "feeds", "hasPart", and "hasPoint".
# The fourth column is the "object" equipment. The fifth column is the "object" equipment's BRICK class.
# Save this Excel table as a ".csv" document. Use Python Pandas to parse this .csv" document.
df=pd.read_csv('Excel_list_of_building_equipment_relations.csv')
print(df)

   subjectURI brick_class  relation          objectURI  \
0        FCU1         FCU     feeds              Zone1   
1        FCU2         FCU     feeds              Zone1   
2        FCU3         FCU     feeds              Zone1   
3        FCU4         FCU     feeds              Zone1   
4        FCU5         FCU     feeds              Zone1   
..        ...         ...       ...                ...   
64   Furnace2         AHU  hasPoint  actualTemperature   
65   Furnace2         AHU  hasPoint     actualHumidity   
66   Furnace2         AHU  hasPoint     rawTemperature   
67   Furnace2         AHU  hasPoint     desiredFanMode   
68   Furnace2         AHU  hasPoint   desiredHeatRange   

                     brick_class.1  
0                        HVAC_Zone  
1                        HVAC_Zone  
2                        HVAC_Zone  
3                        HVAC_Zone  
4                        HVAC_Zone  
..                             ...  
64          Air_Temperature_Sensor  
65     

In [9]:
# Using the Python "re" library, revise the Python Pandas dataset "df" in such a way that for all dataset entries,
# the spaces are replaced with underscores, and all special characters are deleted
def uni_encode(mystring):
    mystring=str(mystring)
    mystring=re.sub(r'[^\w\s]', '', mystring)
    mynewstring=mystring.replace(" ", "_")
    return mynewstring
df=df.applymap(lambda x: uni_encode(x) if pd.notnull(x) else x)

In [10]:
# Using BuildingMOTIF's "model creation" work flow, iterate the Python Pandas dataset (representing the ".csv" document),
# in order to add the BRICK classes for all pieces of equipment
length_dataframe=len(df)
for row_df in range(0,length_dataframe):
    for column_df in [0,3]:
        equipment_name =df.iloc[row_df,column_df] # first and fourth columns
        equipment_brick_class=df.iloc[row_df,column_df+1] # second and fifth columns
        dummy_code = f"equipment_template= brick.get_template_by_name(BRICK.{equipment_brick_class})"
        exec(dummy_code)
        equipment_binding_dict = {"name": BLDG[equipment_name]}
        equipment_graph = equipment_template.evaluate(equipment_binding_dict)
        model.add_graph(equipment_graph)




In [11]:
# Add hasPart, hasPoint, and feeds relationships
for row_df in range(0,length_dataframe):
    equipment_name1 =df.iloc[row_df,0] # first column
    equipment_name2 =df.iloc[row_df,3] # fourth column
    BRICK_relation=df.iloc[row_df,2] # third column
    dummy_code=f"model.graph.add((BLDG['{equipment_name1}'], BRICK.{BRICK_relation}, BLDG['{equipment_name2}']))"
    exec(dummy_code)

In [12]:
# print model to confirm that components were added and connected
print(model.graph.serialize())


@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Bldg> a brick:Building ;
    brick:hasPart <urn:bldg/AdminSpace>,
        <urn:bldg/Office> .

<urn:bldg/FCU1> a brick:FCU ;
    brick:feeds <urn:bldg/Zone1> ;
    brick:hasPoint <urn:bldg/fanLevel>,
        <urn:bldg/mode>,
        <urn:bldg/on>,
        <urn:bldg/targetTemperature>,
        <urn:bldg/temperature>,
        <urn:bldg/timeOfTemperature> .

<urn:bldg/FCU2> a brick:FCU ;
    brick:feeds <urn:bldg/Zone1> ;
    brick:hasPoint <urn:bldg/fanLevel>,
        <urn:bldg/mode>,
        <urn:bldg/on>,
        <urn:bldg/targetTemperature>,
        <urn:bldg/temperature>,
        <urn:bldg/timeOfTemperature> .

<urn:bldg/FCU3> a brick:FCU ;
    brick:feeds <urn:bldg/Zone1> ;
    brick:hasPoint <urn:bldg/fanLevel>,
        <urn:bldg/mode>,
        <urn:bldg/on>,
        <urn:bldg/targetTemperature>,
        <urn:bldg/temperature>,
        <u

In [13]:
#save BRICK ".csv" model as a file named "sample_building_model.ttl"
model.graph.serialize(destination="sample_building_model.ttl")

<Graph identifier=9df97d08-c8b0-4030-8e9e-0dbe29678ed2 (<class 'rdflib.graph.Graph'>)>